# 01 - First Exploration

First exploration of the data.

## Preliminaires 

### System 

In [ ]:
cd ../

In [ ]:
pwd

### Imports

In [ ]:
import os, sys, logging

from dataclasses import dataclass

In [ ]:
from IPython.display import display, HTML

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import missingno as msno

In [ ]:
from gbs.etl.extract import Extract

### Data

In [ ]:
extract = Extract()
extract

In [ ]:
data_dir = os.path.join(
    # os.getcwd(),
    extract.base,
    extract.folder,
    extract.subfolder,
)
data_dir

In [ ]:
# # get data
# if (not os.path.exists(data_dir)) or  (not os.listdir(data_dir)):
#     extract.get_all(clean=True, include_production=True)

In [ ]:
# extract.get_all(clean=True, production=True)

In [ ]:
data_dir = "./data/source/"
os.listdir(data_dir)

In [ ]:
!rm data/source/production.zip

In [ ]:
!tree -L 1 ./

In [ ]:
!tree -L 3 data/

In [ ]:
crops = pd.read_csv(os.path.join(data_dir, "crops.csv"))
crops

In [ ]:
country_specs = pd.read_csv(os.path.join(data_dir, "country_specs.csv"))
country_specs.head()

In [ ]:
_path = "./data/source/production/"

fn_list = [
    os.path.join(_path, f) for f in os.listdir(_path) if f.endswith(".csv")
]
fn_list

In [ ]:
# for fn in fn_list:
#     # read file
#     with open(fn, "r", encoding="latin-1") as f:
#         txt = f.read()

#     # clean file
#     txt.replace(",'", ",")

#     # write file
#     with open(fn, "w", encoding="utf8") as f:
#         f.write(txt)

In [ ]:
_path = "./data/source/production/"

In [ ]:
@dataclass
class Production:
    """Production data for a crop in a country in a year"""

    areacodes = pd.read_csv(os.path.join(_path, "areacodes.csv"))
    data_normalized = pd.read_csv(os.path.join(_path, "data_normalized.csv"))
    flags = pd.read_csv(os.path.join(_path, "flags.csv"))
    itemcodes = pd.read_csv(os.path.join(_path, "itemcodes.csv"))

## Exploration 

### Country Specs 

#### Display

In [ ]:
country_specs.head(10)

In [ ]:
country_specs.tail(10)

In [ ]:
country_specs.sample(10)

#### Structure

In [ ]:
country_specs.shape

In [ ]:
country_specs.columns

In [ ]:
country_specs.dtypes

In [ ]:
country_specs.info()

In [ ]:
crops.dtypes.value_counts()

In [ ]:
for dtype in ["object", "float", "int"]:
    selected_dtype = country_specs.select_dtypes(include=[dtype])
    display(selected_dtype.columns)

In [ ]:
_num = country_specs.select_dtypes(include=["number"])
_num

In [ ]:
_num_cols = _num.columns.tolist()
[i for i in _num_cols if ("code" not in i) and ("id" not in i)]

**Conclusion**

No data in the crop dataset.

#### Nan

In [ ]:
crops.isna().sum()

In [ ]:
tmp = crops.isna().mean().round(2)
tmp[tmp > 0.00]

In [ ]:
len(tmp[tmp > 0.00]) / len(tmp)

In [ ]:
tmp = crops.isna().mean(axis=1).round(2)
tmp.value_counts().sort_index()

In [ ]:
msno.matrix(country_specs)

**Conclusion**

- delete is_EU27 and is_south_america

In [ ]:
# delete is_EU27 and is_south_america

In [ ]:
drop_cols = ["is_EU27", "is_south_america"]
country_specs = country_specs.drop(columns=drop_cols)
msno.matrix(country_specs)

In [ ]:
tmp = country_specs.isna().mean(axis=1).round(2)
tmp.value_counts().sort_index()

In [ ]:
threshold = 0.2
tmp[tmp >= threshold]

In [ ]:
drop_idxs = tmp[tmp > threshold].index
country_specs.loc[drop_idxs, :]

**Conclusion**

- countries with Nan rate > 0.3 are Island or -100k pop (Monaco, Antigua)

In [ ]:
country_specs = country_specs.drop(
    index=drop_idxs, columns=drop_cols, errors="ignore"
)
msno.matrix(country_specs)

In [ ]:
num_cols = country_specs.select_dtypes(
    include=["float", "int"]
).columns.tolist()

num_cols = [i for i in num_cols if "code" not in i]

categ_cols = [
    "alpha_3_code",
    "FAO_country_name",
    "exiobase_region_name",
    "globio_country_code",
    "globio_country_name",
    "USS30_region_name",
]


country_specs = country_specs.loc[:, num_cols + categ_cols]

In [ ]:
country_specs

**Conclusion**

- No relevant data in the crop dataset

#### Data Inspection

In [ ]:
country_specs.FAO_country_name.value_counts()

In [ ]:
country_specs.FAO_country_name.nunique()

In [ ]:
country_specs.USS30_region_name.value_counts()

In [ ]:
country_specs.FAO_country_name.value_counts().value_counts()

In [ ]:
country_specs.groupby("exiobase_region_name").FAO_country_name.count()

#### DataSet Conclusion

- No relevant data in the crop dataset

### Crops 

#### Wheat Selection

In [ ]:
feature = "Wheat"
crops = crops.loc[crops.item_name == feature, :]

In [ ]:
item_cols = [i for i in crops.columns if "item" in i]
item_cols

In [ ]:
crops.drop(columns=item_cols, inplace=True, errors="ignore")
crops

In [ ]:
crops

#### Display

In [ ]:
crops.head(10)

In [ ]:
crops.tail(10)

In [ ]:
crops.sample(10)

#### Structure

In [ ]:
crops.drop(columns="id", inplace=True, errors="ignore")

In [ ]:
crops.shape

In [ ]:
crops.columns

In [ ]:
crops.dtypes

In [ ]:
crops.info()

In [ ]:
crops.dtypes.value_counts()

In [ ]:
for dtype in ["object", "float", "int"]:
    selected_dtype = crops.select_dtypes(include=[dtype])
    display(selected_dtype.columns)

In [ ]:
_num = crops.select_dtypes(include=["number"])
_num

In [ ]:
_feat_cols = [i for i in _num.columns if "msa" in i]
_feat_cols

#### Separation Static/Dynamic

In [ ]:
categ_cols = [i for i in crops.columns if "msa" not in i]
static_cols = [i for i in crops.columns if "static" in i]
dynamic_cols = [i for i in crops.columns if "dynamic" in i]

display(categ_cols)
display(static_cols)
display(dynamic_cols)

In [ ]:
crops_static = crops.loc[:, categ_cols + static_cols]
crops_static.head()

In [ ]:
crops_dynamic = crops.loc[:, categ_cols + dynamic_cols]
crops_dynamic

# BE CAREFULL => in the report we do have to distinguish between terestrial static, dynamic and marine static, dynamic

#### Summize

crops_static

In [ ]:
crops_static

In [ ]:
sum_static = crops_static.iloc[:, 1:].sum(axis=1)
sum_static

In [ ]:
crops_static = crops_static.iloc[:, :1]
crops_static["sum_static"] = sum_static.values

#### Data Inspection

In [ ]:
crops_static.describe()

In [ ]:
crops.globio_country_code.value_counts()

In [ ]:
crops.globio_country_code.nunique()

**Conclusion**

????

#### Merge Country Specs and crops

In [ ]:
country_specs

In [ ]:
# merged = pd.merge(left=_country_specs, right=crops, on="globio_country_code",how="left")
# merged

In [ ]:
tmp = country_specs.loc[:, ["globio_country_code", "globio_country_name"]]
tmp.index = tmp.globio_country_code
tmp.drop(columns="globio_country_code", inplace=True)
tmp.index.name = None
tmp = tmp.to_dict().get("globio_country_name")
tmp

In [ ]:
crops_static["globio_country_name"] = crops_static.globio_country_code.apply(
    lambda i: tmp.get(i, np.nan)
)
crops_static

#### Nan

In [ ]:
crops_static.isna().sum()

**Conclusion**

???

**Conclusion**
???

## Production

#### Table Analysis

In [ ]:
Production.areacodes

In [ ]:
Production.flags

In [ ]:
flags = {
    k: v
    for k, v in zip(
        Production.flags.Flag.values, Production.flags.Description.values
    )
}
flags

In [ ]:
Production.itemcodes

In [ ]:
data = Production.data_normalized
data

In [ ]:
data["Flag_value"] = data.Flag.apply(lambda i: flags[i])
data

In [ ]:
data.columns

#### Feature selection

In [ ]:
cols = [
    # "Area Code",
    # "Area Code (M49)",
    "Area",
    # "Item Code",
    # "Item Code (CPC)",
    "Item",
    # "Element Code",
    "Element",
    # "Year Code",
    "Year",
    "Unit",
    "Value",
    # "Flag",
    # "Note",
    "Flag_value",
]

In [ ]:
data = data.loc[:, cols]
data

In [ ]:
data.Element.nunique()

In [ ]:
data.Element.value_counts()

In [ ]:
data.Item.value_counts()

In [ ]:
data_weat = data.loc[data.Item.str.lower().str.contains("wheat"), :]
data_weat

In [ ]:
data_weat_2019 = data_weat.loc[data_weat.Year == 2019, :]
data_weat_2019

In [ ]:
data_weat_2019_h = data_weat_2019.loc[data_weat_2019.Unit == "ha", :]

In [ ]:
data_weat_2019_h

In [ ]:
data_weat_2019_h.columns

In [ ]:
cols = [
    "Area",
    # "Item",
    # "Element",
    # "Year",
    # "Unit",
    "Value",
    "Flag_value",
]

data_weat_2019_h = data_weat_2019_h.loc[:, cols]
data_weat_2019_h

#### Display 

In [ ]:
data_weat_2019_h.head(10)

In [ ]:
data_weat_2019_h.tail(10)

In [ ]:
data_weat_2019_h.sample(10)

#### Structure

In [ ]:
data_weat_2019_h.shape

In [ ]:
data_weat_2019_h.info()

#### Nan

In [ ]:
data_weat_2019_h.isna().sum()

In [ ]:
data_weat_2019_h

data_weat_2019_h.shape

## Final Merge

### Keys Analysis

In [ ]:
data_weat_2019_h.sort_values("Area", inplace=True, ascending=True)
data_weat_2019_h.rename(
    columns={"Value": "km2"}, inplace=True, errors="ignore"
)
data_weat_2019_h

####### BE CARREFULLL KM2 is supposed

In [ ]:
crops_static.sort_values("globio_country_name", ascending=True, inplace=True)
crops_static

In [ ]:
country_specs.sort_values("FAO_country_name", ascending=True, inplace=True)
country_specs

In [ ]:
crops_static.shape

In [ ]:
data_weat_2019_h.shape

In [ ]:
country_specs.shape

### data vs country ON FAO_country_name

In [ ]:
merge1 = pd.merge(
    left=country_specs,
    right=data_weat_2019_h,
    left_on="FAO_country_name",
    right_on="Area",
    how="outer",
    indicator=True,
)

merge1

In [ ]:
merge1.rename(columns={"_merge": "_merge_1"}, inplace=True)
merge1

data_weat_2019_h.head()

### Merge1 v crops 

In [ ]:
merge2 = pd.merge(
    left=merge1,
    right=crops_static,
    left_on="globio_country_name",
    right_on="globio_country_name",
    how="outer",
    indicator=True,
)

merge2

In [ ]:
merge2.rename(columns={"_merge": "_merge_2"}, inplace=True)
merge2

In [ ]:
merge2.loc[merge2.loc[:, "_merge_2"] == "both", :]

In [ ]:
merge2.loc[merge2.loc[:, "_merge_2"] == "both", :]

In [ ]:
final = merge2.loc[merge2.loc[:, "_merge_2"] == "both", :]

### Select Features 

In [ ]:
final

In [ ]:
final.columns

In [ ]:
cols = [
    # "exiobase_region_id",
    "alpha_3_code",
    "FAO_country_name",
    # "exiobase_region_name",
    # "globio_country_code_x",
    "globio_country_name",
    "USS30_region_name",
    "Area",
    "km2",
    "Flag_value",
    # "_merge_1",
    # "globio_country_code_y",
    "sum_static",
    # "_merge_2",
]

In [ ]:
final = final.loc[:, cols]
final

In [ ]:
final["ms.km2"] = final.km2 * final.sum_static
final

## Feature Engineering

### population

In [ ]:
fn = "./data/source/population/API_SP.POP.TOTL_DS2_en_csv_v2_6298256.csv"


encoding = "latin-1"
encoding = "utf8"
with open(fn, "r", encoding=encoding) as f:
    txt = f.readlines()

In [ ]:
txt

In [ ]:
txt[4:]

In [ ]:
with open(fn, "w", encoding="utf8") as f:
    # txt[4:].write(f)
    f.writelines(txt[4:])

In [ ]:
pop = pd.read_csv(fn)

In [ ]:
pop

In [ ]:
pop = pop.loc[:, pop.columns.tolist()[:2] + ["2019"]]
pop

In [ ]:
pop.rename(columns={"2019": "population"}, inplace=True)

## Gpd

In [ ]:
fn = "./data/source/gpd/API_NY.GDP.PCAP.CD_DS2_en_csv_v2_6298251.csv"

In [ ]:
with open(fn, "r", encoding="utf8") as f:
    txt = f.readlines()

In [ ]:
txt

In [ ]:
txt[4:]

In [ ]:
with open(fn, "w", encoding="utf8") as f:
    f.writelines(txt[4:])

In [ ]:
gpd = pd.read_csv(fn)
gpd

In [ ]:
gpd = gpd.loc[:, gpd.columns.tolist()[:2] + ["2019"]]
gpd

In [ ]:
gpd.rename(columns={"2019": "gpd_per_capita"}, inplace=True)
gpd

### merge both

In [ ]:
pop_gpd = pd.merge(
    left=pop, right=gpd, on=["Country Name", "Country Code"], how="outer"
)
pop_gpd

In [ ]:
test_3_codes = ["AUT", "BEL, BGR"]

pop_gpd.loc[pop_gpd.loc[:, "Country Name"].str.contains("Bel"), :]

In [ ]:
pop_gpd.loc[pop_gpd.loc[:, "Country Name"].str.contains("Bul"), :]

In [ ]:
final_gpd_pop = pd.merge(
    left=final,
    right=pop_gpd,
    left_on="alpha_3_code",
    right_on="Country Code",
    how="left",
)
final_gpd_pop

### Final Selection

In [ ]:
final_gpd_pop.columns

In [ ]:
cols = [
    # "exiobase_region_id",
    # "alpha_3_code",
    # "FAO_country_name",
    # "exiobase_region_name",
    # "globio_country_code_x",
    # "globio_country_name",
    "USS30_region_name",
    # "Area",
    "km2",
    # "Flag_value",
    # "_merge_1",
    # "globio_country_code_y",
    "sum_static",
    # "_merge_2",
    "ms.km2",
    "Country Name",
    "Country Code",
    "population",
    "gpd_per_capita",
]

In [ ]:
final_gpd_pop = final_gpd_pop.loc[:, cols]
final_gpd_pop

In [ ]:
final_gpd_pop.columns = [i.lower() for i in final_gpd_pop.columns]
final_gpd_pop

In [ ]:
final_gpd_pop.columns

In [ ]:
cols = [
    "uss30_region_name",
    "country name",
    "country code",
    "km2",
    "sum_static",
    "ms.km2",
    "population",
    "gpd_per_capita",
]

In [ ]:
final_gpd_pop = final_gpd_pop.loc[:, cols]
final_gpd_pop.rename(
    columns={
        "uss30_region_name": "region",
        "sum_static": "sum_msa_static",
        "ms.km2": "msa.km2",
    }
)

In [ ]:
final_gpd_pop.to_csv("./data/final.csv", index=False)